# Setup

lsgraph was created with the aim of supporting reskilling and upskilling efforts for both individuals and organisations. lsgraph provides services to: 

* collect course details and associate with skills, 
* recommend courses for a learner, 
* recommend job profiles, 
* suggest reskilling options across a workforce.

This notebook walks through the process of setting up skills graphs, organisations, profiles and users.

In [1]:
import json
import os
import pandas as pd
import requests

# If using the docker-compose file lsgraph will be available on localhost port 5000
base_url = "http://localhost:5000"

# Create graph

A skills graph is a linked collection of skills that aids navigation and disambiguates skill names with multiple meanings. 

Within the sample data directory is a graph in nested dictionary format. 

In [2]:
with open('sample_data/graph.json') as fp:
    skill_graph = json.load(fp)
    
# View top level skills
print("The top level skills are:", skill_graph.keys())

The top level skills are: dict_keys(['personal skills', 'art & design', 'certifications', 'data science', 'software engineering', 'information technology', 'engineering', 'natural sciences', 'languages', 'business & management', 'social sciences', 'mathematics', 'computer science'])


In [3]:
# Create a new skills graph in lsgraph

graph_tag = "test"

new_graph = {"name":"Test graph",
             "graph_tag":graph_tag,
             "description":"",
             "graph":skill_graph,
            }

r = requests.post(base_url + "/api/v1/graphs", json=new_graph)

print("Status code", r.status_code)

print("Response", r.json())
      
graph_id = r.json()["id"]

Status code 200
Response {'id': 1}


In [4]:
# View all graphs

r = requests.get(base_url + "/api/v1/graphs")
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'graph_tag': 'test', 'id': 1, 'name': 'Test graph'}]


Detailed information on a skills graph can be downloaded.

This is important for accessing the ids of skills. The same nested dictionary format used to load the skills graph is also returned with the addition of a \_meta key that contains the id in a dictionary.

The skills graph can be duplicated by calling /graphs where the \_meta key will be ignored.

In [5]:
r = requests.get(base_url + "/api/v1/graphs/{0}".format(graph_id))
print("Status code", r.status_code)

skill_graph = r.json()["graph"]

Status code 200


# Create organisation

An organisation is a collection of users that share a skills graph and profiles.

In [6]:
# Create a new organisation

new_org = {"name":"Test organisation",
             "graph":graph_tag,
            }

r = requests.post(base_url + "/api/v1/organisations", json=new_org)
print("Status code", r.status_code)
print("Response", r.json())

organisation_id = r.json()["id"]

Status code 200
Response {'id': 1}


In [7]:
# View all organisations

r = requests.get(base_url + "/api/v1/organisations")
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'id': 1, 'name': 'Test organisation'}]


In [8]:
# View more detail on one organisation

r = requests.get(base_url + "/api/v1/organisations/{0}".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response {'created_at': 'Sun, 28 Mar 2021 20:52:44 GMT', 'graph': 'test', 'id': 1, 'name': 'Test organisation', 'updated_at': 'Sun, 28 Mar 2021 20:52:44 GMT'}


# Create profiles

Profiles are used to collect skills and associate levels.

A variety of sample profiles for specific jobs are available

In [9]:
# Load sample data

profile_dir = 'sample_data/profiles/'

level_map = {'B':'beginner','I':'intermediate','A':'advanced','E':'expert'}

def skill_id_map(graph, path):
    if len(path) == 1:
        return graph[path[0]]['_meta']['id']
    return skill_id_map(graph[path[0]], path[1:])

profiles = {}

for fn in os.listdir(profile_dir):
    profile_name = fn[:-len('.skills.csv')]
    data = pd.read_csv(os.path.join(profile_dir, fn), header=None)
    profile_competences = {}
    for _,competence in data.iterrows():
        level = competence[0]
        path=[i for i in competence[1:].tolist() if isinstance(i, str)]
        skill_id = skill_id_map(skill_graph, path)
        profile_competences[skill_id] = level_map[level]
    profiles[profile_name] = profile_competences

In [10]:
# Load all profiles

for profile_name, competences in profiles.items():
    print("Creating profile", profile_name)
    new_profile = {"name":profile_name,
                   "competences":competences
                  }
    r = requests.post(base_url + "/api/v1/organisations/{0}/profiles".format(organisation_id), json=new_profile)
    if r.status_code != 200:
        print("Status code", r.status_code)
        print("Error with", profile_name)
        break

Creating profile Applications Engineer
Creating profile Data Scientist
Creating profile Computer and Information Systems Managers
Creating profile Web Developers
Creating profile Software Engineer
Creating profile Software Engineer ML
Creating profile Systems Engineer
Creating profile DevOps Engineer
Creating profile Machine Learning Researcher
Creating profile Business Intelligence Analyst
Creating profile Software Developers, Applications
Creating profile Cloud architect
Creating profile Sales Engineer
Creating profile Database Administrators
Creating profile Security Engineer
Creating profile Computer Hardware Engineers
Creating profile Data Engineer
Creating profile Product Manager
Creating profile UX Designer
Creating profile Solutions Architect
Creating profile Mobile Developer
Creating profile Machine Learning Engineer
Creating profile Information Security Analysts
Creating profile Data Analyst
Creating profile Full stack developer
Creating profile Systems Administrator
Creating

In [11]:
# List all profiles for an organisation

r = requests.get(base_url + "/api/v1/organisations/{0}/profiles".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'id': 1, 'name': 'Applications Engineer'}, {'id': 2, 'name': 'Data Scientist'}, {'id': 3, 'name': 'Computer and Information Systems Managers'}, {'id': 4, 'name': 'Web Developers'}, {'id': 5, 'name': 'Software Engineer'}, {'id': 6, 'name': 'Software Engineer ML'}, {'id': 7, 'name': 'Systems Engineer'}, {'id': 8, 'name': 'DevOps Engineer'}, {'id': 9, 'name': 'Machine Learning Researcher'}, {'id': 10, 'name': 'Business Intelligence Analyst'}, {'id': 11, 'name': 'Software Developers, Applications'}, {'id': 12, 'name': 'Cloud architect'}, {'id': 13, 'name': 'Sales Engineer'}, {'id': 14, 'name': 'Database Administrators'}, {'id': 15, 'name': 'Security Engineer'}, {'id': 16, 'name': 'Computer Hardware Engineers'}, {'id': 17, 'name': 'Data Engineer'}, {'id': 18, 'name': 'Product Manager'}, {'id': 19, 'name': 'UX Designer'}, {'id': 20, 'name': 'Solutions Architect'}, {'id': 21, 'name': 'Mobile Developer'}, {'id': 22, 'name': 'Machine Learning Engineer'}, {'id': 23, 'n

# Create users

In [12]:
# Create four users with the competences associated with a job profile

users = [("Adam", profiles["Data Engineer"]), 
         ("Beth", profiles["Electrical Engineer"]), 
         ("Chris", profiles["Mobile Developer"]), 
         ("Dani", profiles["Security Engineer"])]

for user,competences in users:
    print("Creating user", user)
    new_user = {"email":"{0}@test.local".format(user),
                "name":user,
                "graph":graph_tag,
                "competences":competences
               }
    r = requests.post(base_url + "/api/v1/organisations/{0}/users".format(organisation_id), json=new_user)
    print("Status code", r.status_code)
    print("Response", r.json())

Creating user Adam
Status code 200
Response {'id': 1}
Creating user Beth
Status code 200
Response {'id': 2}
Creating user Chris
Status code 200
Response {'id': 3}
Creating user Dani
Status code 200
Response {'id': 4}


In [13]:
# List all users for an organisation

r = requests.get(base_url + "/api/v1/organisations/{0}/users".format(organisation_id))
print("Status code", r.status_code)

print("Response", r.json())

Status code 200
Response [{'id': 1, 'name': 'Adam'}, {'id': 2, 'name': 'Beth'}, {'id': 3, 'name': 'Chris'}, {'id': 4, 'name': 'Dani'}]
